In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv('C:/Users/baek/anaconda3/practice/two_day/ratings_train.txt', sep = "\t", encoding = "utf-8")
test_df = pd.read_csv('C:/Users/baek/anaconda3/practice/two_day/ratings_test.txt', sep = "\t", encoding = "utf-8")
print(train_df.head(3))
print(test_df.head(3))

         id                           document  label
0   9976970                아 더빙.. 진짜 짜증나네요 목소리      0
1   3819312  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2  10265843                  너무재밓었다그래서보는것을추천한다      0
        id                                document  label
0  6270596                                     굳 ㅋ      1
1  9274899                    GDNTOPCLASSINTHECLUB      0
2  8544678  뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아      0


In [3]:
train_df['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [4]:
test_df['label'].value_counts()

1    25173
0    24827
Name: label, dtype: int64

In [5]:
import re

train_df = train_df.fillna(' ')
# 정규 표현식을 이용하여 숫자를 공백으로 변경(정규 표현식으로 \d 는 숫자를 의미함.) 
train_df['document'] = train_df['document'].apply( lambda x : re.sub(r"\d+", " ", x) )

# 테스트 데이터 셋을 로딩하고 동일하게 Null 및 숫자를 공백으로 변환
test_df = test_df[:300]
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply( lambda x : re.sub(r"\d+", " ", x) )

train_df.drop('id', axis=1, inplace=True) 
test_df.drop('id', axis=1, inplace=True)

In [6]:
import warnings
warnings.simplefilter("ignore")

import konlpy
# import tweepy
# from tweepy import StreamListener
# from tweepy import Stream
konlpy.__version__

'0.5.2'

In [9]:
from konlpy.tag import *

hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
#ecab = Mecab()
okt = Okt()

In [10]:
#from konlpy.tag import Kkma

twitter = okt
def tw_tokenizer(text):
    # 입력 인자로 들어온 text 를 형태소 단어로 토큰화 하여 list 객체 반환
    tokens_ko = twitter.morphs(text)
    return tokens_ko

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Twitter 객체의 morphs( ) 객체를 이용한 tokenizer를 사용. ngram_range는 (1,2). 최소단위는 3번 
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3)


In [12]:
tfidf_vect.fit(train_df['document'])

TfidfVectorizer(min_df=3, ngram_range=(1, 2),
                tokenizer=<function tw_tokenizer at 0x0000019792FC88B0>)

In [13]:
tfidf_matrix_train = tfidf_vect.transform(train_df['document'][:100])

In [14]:
# Logistic Regression 을 이용하여 감성 분석 Classification 수행. 
lg_clf = LogisticRegression(random_state=0)

# Parameter C 최적화를 위해 GridSearchCV 를 이용. C 지정해주기
params = { 'C': [0.5, 1, 5, 10 ] }
grid_cv = GridSearchCV(lg_clf , param_grid=params , cv=3 ,scoring='accuracy', verbose=1 )
grid_cv.fit(tfidf_matrix_train , train_df['label'][:100] )
print(grid_cv.best_params_ , round(grid_cv.best_score_,4))

Fitting 3 folds for each of 4 candidates, totalling 12 fits
{'C': 0.5} 0.5401


In [15]:
from sklearn.metrics import accuracy_score

# 학습 데이터를 적용한 TfidfVectorizer를 이용하여 테스트 데이터를 TF-IDF 값으로 Feature 변환함. 
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

# classifier 는 GridSearchCV에서 최적 파라미터로 학습된 classifier를 그대로 이용
best_estimator = grid_cv.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)

print('Logistic Regression 정확도: ',accuracy_score(test_df['label'],preds))

Logistic Regression 정확도:  0.5366666666666666
